# Ensayo clínico de la vacuna covid

Ejemplo simplificado de un test de hipótesis aplicado para decidir si una vacuna es eficaz en base a los resultados de un ensayo clínico. 

El test clínico se realiza sobre un grupo de 10.000 personas aproximadamente. A la mitad de estas personas se les administra la vacuna a testear y a la otra mitad se les inyecta un placebo.  Si una persona fué vacunada se denomina que pertenece al *grupo tratamiento* y si recibió placebo al *grupo control*. Los participantes del ensayo ignoran si fueron vacunados o fueron inoculados con placebo. 

De acuerdo pasa el tiempo desde el inicio del ensayo, algunos de los particpantes se van contagiando de covid. Para terminar el ensayo se espera que cierto número de participantes prestablecido se contagie covid. Este número es el tamaño de la muestra del test: 

In [ ]:
sample_size = 150

## Probabilidad de tratamiento

Entre los contagiados algunos pertenecen al grupo tratamiento (vacunados) y otros al control (placebo). Si la vacuna tiene eficacia nula, debería haber cantidades similares de contagiados en el grupo tratamiento y control. Si la vacuna es muy eficaz, pocos de los contagiados son del grupo tratamiento y la mayoría proviene del grupo control.   

Para cada contagiado a hay dos resultados posibles:   

- Pertenece al grupo tratamiento
- Pertenece al grupo control  
  
En base a esto definimos una variable de Bernoulli que toma el valor 1 si el contagiado pertenece al grupo de tratamiento y 0 si pertenece al grupo control. La probabilidad que un contagiado pertenezca al grupo tratamiento se puede calcular a partir de la eficacia de la vacuna:

In [ ]:
def get_treatment_probability(efficacy):
    return (1-efficacy)/(2-efficacy)

## Hipótesis nula

Construimos las hipótesis en base a la eficacia de la vacuna. Para la hipótesis nula consideramos la eficacia máxima para la que vamos a rechazar la vacuna. Este es un valor bajo aunque menos conservador que tomar eficacia nula.

In [ ]:
efficacy0 = 0.3

Según la hipotésis nula, la probabilidad que un contagiado pertenezca al grupo tratamiento es,

In [ ]:
probability0 = get_treatment_probability(efficacy0)
probability0

## Hipótesis alternativa

Para la hipótesis alternativa tomamos una eficacia mínima para aceptar la vacuna  

In [ ]:
efficacy1 = 0.6

La probabilidad correspondiente es,

In [ ]:
probability1 = get_treatment_probability(efficacy1)
probability1

## Probabilidad de tratamiento vs. eficacia

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
fig, ax = plt.subplots()
ax.set_xlabel("Efficacy")
ax.set_ylabel("Treatment probability")
efficacy = np.linspace(0, 1, 100)
treatment_probability = get_treatment_probability(efficacy)
ax.plot(efficacy, treatment_probability)
ax.vlines(x=efficacy0, ymin=0, ymax=probability0, ls='--', color='tab:orange', label="$H_0$")
ax.hlines(y=probability0, xmin=0, xmax=efficacy0, ls='--', color='tab:orange')
ax.vlines(x=efficacy1, ymin=0, ymax=probability1, ls='--', color='tab:green', label="$H_1$")
ax.hlines(y=probability1, xmin=0, xmax=efficacy1, ls='--', color='tab:green')
ax.set_xlim(left=0)
ax.set_ylim(bottom=0)
ax.legend()

Como la eficacia de la hipótesis alternativa es mayor a la de la nula, la probabilidad que un contagiado pertenezca al grupo tratamiento es menot. 

## Estadístico de test

El estadístico del test es el número de personas contagiadas que pertenecen al grupo tratamiento ($t$). En el mejor caso ninguno de los contagiados pertenece al grupo tratamiento y la situación opuesta todos los contagiados pertenecen al grupo tratamiento. Cubrimos todo el rango entre estos dos extremos:  

In [ ]:
ntreatment = np.arange(0, sample_size+1)
ntreatment

El estadístico de prueba es una variable binomial ya que consiste en un número de pruebas independientes dado por el tamaño de la muestra, dónde cada prueba corresponde a una variable de Bernoulli.

Si la hipótesis nula es verdadera, el estadístico del test tiene una probabilidad de Bernoulli $p_0$:

In [ ]:
import scipy
random_variable0 = scipy.stats.binom(sample_size, probability0)
ntreatment_pmf0 = random_variable0.pmf(ntreatment)

Si en cambio la hipótesis alternativa es verdades, el estadístico de prueba tiene una probabilidad de Bernoulli $p_1$: 

In [ ]:
random_variable1 = scipy.stats.binom(sample_size, probability1)
ntreatment_pmf1 = random_variable1.pmf(ntreatment)

In [ ]:
fig, ax = plt.subplots()
ax.set_xlabel("# Treatment")
ax.set_ylabel("Probability")
ax.plot(ntreatment, ntreatment_pmf0, ls='', marker='o', label="$H_0$")
ax.plot(ntreatment, ntreatment_pmf1, ls='', marker='o', label="$H_1$")
ax.legend()

## Lema de Neyman-Pearson

In [ ]:
pdf_ratio = ntreatment_pmf1 / ntreatment_pmf0
fig, ax = plt.subplots()
ax.set_xlabel("# Treatment")
ax.set_ylabel("$f_1(t)/f_0(y)$")
ax.plot(ntreatment, pdf_ratio, ls='', marker='o')
ax.set_yscale('log')

Seleccionamos para la región crítica los valores del estadístico de test $t$ tal que el cociente $f_1(x)/f_0(x)$ sea mayor que alguna constante k. Como este cociente es decreciente, fijamos la región crítica con $t$ menor que algún $t_c$. Calculamos $t_c$ en base al nivel de confianza del test.

## Valor crítico

Establecemos la significancia del test (probabilidad de cometer un error del tipo I)

In [ ]:
test_significance = 0.05
confidence_level = 1 - test_significance
confidence_level

La región crítica está en la cola inferior de la PMF de la hipótesis nula ya que según la hipótesis alternativa hay menos contagiados en el grupo tratamiento. (test de cola inferior).

 Calculamos el cuantil de $t$ según la hipótesis nula  

In [ ]:
quantile = random_variable0.ppf(test_significance)
quantile

Como el estadístico $t$ es discreto el cuantil corresponde a una probabilidad mayor a la significancia 

In [ ]:
random_variable0.cdf(quantile)

In [ ]:
critical_value = quantile - 1
critical_value

Para ser estrictos calculamos un valor crítico $t_c$ menor al cuantil    

De esta forma si el resultado del test coincide con el cuantil no rechazamos $H_0$, pedimos un valor más extremo para esto.

La región crítica corresponde a $ t \le t_c$ y la región de aceptancia $t_c < t$. Debido a la discretización de $t$, la significancia del test es menor al valor nominal:

In [ ]:
random_variable0.cdf(critical_value)

Aunque no lo necesitamos usar, por completitud calculamos k del lema de Neyman-Pearson. Para calcular la región crítica y el poder del test basta con saber que el cociente de las PDFs es decreciente.

In [ ]:
k = random_variable1.pmf(critical_value) / random_variable0.pmf(critical_value)
k

## Poder del test

El poder del test es la probabilidad de rechazar la hipótesis nula si la hipótesis alternativa es verdadera. En este ejemplo, el poder del test es la probabilidad que $t$ esté en la región crítica (rechazamos la hipótesis nula) si tiene una probabilidad de Bernoulli $p_1$ (la hipótesis alternativa es verdadera):

In [ ]:
test_power = random_variable1.cdf(critical_value)
test_power

Probabilidad de cometer un error del tipo II

In [ ]:
beta = 1 - test_power
beta

Tener un test poderoso es muy deseable ya que tiene alta probabilidad de rechazar la hipótesis nula que es el resultado buscado. En este ejemplo rechazar la hipótesis nula significa rechazar la hipótesis que la eficiencia de la vacuna es baja y, por lo tanto, concluir que la eficiencia de la vacuna es satisfactoria.

In [ ]:
fig, ax = plt.subplots()
ax.set_xlabel("Treatment")
ax.set_ylabel("Probability")
ax.plot(ntreatment, ntreatment_pmf0, ls='', marker='o', label="$H_0$")
ax.plot(ntreatment, ntreatment_pmf1, ls='', marker='o', label="$H_1$")
ax.axvline(critical_value, ls='--', color='tab:gray')
ax.set_xlim(left=25, right=125)
plt.axvspan(0, critical_value, color='tab:gray', alpha=0.25, label="Critical region")
ax.legend()
# plt.savefig("vacuna_covid.svg")

La figura muestra que en la región crítica (área sombreada) la probabilidad bajo la PMF de $H_0$ es la significancia y la probabilidad bajo la PMF de $H_1$ es el poder. Como las dos PMFs están 'separadas', es posible tener un test de baja significancia y alto poder al mismo tiempo. 

Proponemos correr este cuaderno con diferentes número de contagiados y ver como, a significancia fija, varía el poder del test. 

Para una muestra de menos contagiados, el poder del test disminuye, y si se seleccionan más contagiados, el poder aumenta. Este factor compite con el costo dado por el número de participantes y el tiempo que debe correr el ensayo clínico hasta alcanzar el número de contagios deseados. Para maximizar el número de contagiados se hacen los ensayos en poblaciones con alta tasa de contagio.